# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")

In [3]:
cursor = conn.cursor()
cursor.execute("UPDATE movie SET mid=LTRIM(RTRIM(mid)),year=SUBstr(RTRIM(year),-4,4)")
cursor.execute("UPDATE person SET pid=LTRIM(RTRIM(pid)),name=LOWER(LTRIM(RTRIM(name)))")
cursor.execute("UPDATE m_director SET mid=LTRIM(RTRIM(mid)),pid=LTRIM(RTRIM(pid))")
cursor.execute("UPDATE m_genre SET mid=LTRIM(RTRIM(mid))")
cursor.execute("UPDATE m_cast SET mid=LTRIM(RTRIM(mid)),pid=LTRIM(RTRIM(pid))")
cursor.execute("UPDATE m_producer SET mid=LTRIM(RTRIM(mid)),pid=LTRIM(RTRIM(pid))")
conn.commit()

In [4]:
result=pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

In [5]:
result['name']

0          Movie
1          Genre
2       Language
3        Country
4       Location
5     M_Location
6      M_Country
7     M_Language
8        M_Genre
9         Person
10    M_Producer
11    M_Director
12        M_Cast
Name: name, dtype: object

In [6]:
result=pd.read_sql_query("SELECT * FROM movie LIMIT 1", conn)
print(result)
print('===genere'*10)
result=pd.read_sql_query("SELECT * FROM genre LIMIT 1", conn)
print(result)
print('===person'*10)
result=pd.read_sql_query("SELECT * FROM person LIMIT 2", conn)
print(result)
print('===m_director'*10)
result=pd.read_sql_query("SELECT * FROM m_director LIMIT 2", conn)
print(result)
print('===m_genere'*10)
result=pd.read_sql_query("SELECT * FROM m_genre LIMIT 2", conn)
print(result)
print('===m_cast'*10)
result=pd.read_sql_query("SELECT * FROM m_cast LIMIT 2", conn)
print(result)
print('===producert'*10)
result=pd.read_sql_query("SELECT * FROM m_producer LIMIT 2", conn)
print(result)

   index        MID   title  year  rating  num_votes
0      0  tt2388771  Mowgli  2018     6.6      21967
===genere===genere===genere===genere===genere===genere===genere===genere===genere===genere
   index                                   Name  GID
0      0  Adventure, Drama, Fantasy                0
===person===person===person===person===person===person===person===person===person===person
   index        PID            Name  Gender
0      0  nm0000288  christian bale    Male
1      1  nm0000949  cate blanchett  Female
===m_director===m_director===m_director===m_director===m_director===m_director===m_director===m_director===m_director===m_director
   index        MID        PID  ID
0      0  tt2388771  nm0785227   0
1      1  tt5164214  nm0002657   1
===m_genere===m_genere===m_genere===m_genere===m_genere===m_genere===m_genere===m_genere===m_genere===m_genere
   index        MID  GID  ID
0      0  tt2388771    0   0
1      1  tt5164214    1   1
===m_cast===m_cast===m_cast===m_cast===m

## Sample Code

In [7]:
%%time
# Write your sql query below

query = """
        SELECT TRIM(Movie.title) AS 'Movie_Name'
        FROM Movie
        WHERE Movie.rating < 3
        
        """

q = pd.read_sql_query(query, conn)    
print(q.shape)
q.head()

(85, 1)
Wall time: 4.99 ms


## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [8]:
%%time
# Write your sql query below

query = """
        SELECT p.name,m.title,m.year,g.name From movie m 
        LEFT JOIN m_director d ON m.mid=d.mid 
        LEFT join person P ON p.pid=d.pid 
        LEFT JOIN m_genre mg ON mg.mid=m.mid
        LEFT JOIN genre g ON g.gid=mg.gid
        WHERE  g.Name LIKE '%Comedy%' AND CAST(m.year AS INTEGER) %4 ==0
        
        """

q1 = pd.read_sql_query(query, conn)    
print(q1.shape)

(232, 4)
Wall time: 88.8 ms


In [9]:
q1.head()

,Name,title,year,Name
0,milap zaveri,Mastizaade,2016,Comedy
1,danny leiner,Harold & Kumar Go to White Castle,2004,"Adventure, Comedy"
2,anurag kashyap,Gangs of Wasseypur,2012,"Action, Comedy, Crime"
3,frank coraci,Around the World in 80 Days,2004,"Action, Adventure, Comedy"
4,griffin dunne,The Accidental Husband,2008,"Comedy, Romance"



## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [10]:
%%time
# Write your sql query below

query = """
        SELECT p.name,m.title,m.year FROM movie m 
        LEFT JOIN m_cast mc ON m.mid=mc.mid
        LEFT JOIN person p ON p.pid=mc.pid
        WHERE m.title= 'Anand'    
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)

(17, 3)
Wall time: 220 ms


In [11]:
q2.head()

,Name,title,year
0,amitabh bachchan,Anand,1971
1,rajesh khanna,Anand,1971
2,brahm bhardwaj,Anand,1971
3,ramesh deo,Anand,1971
4,seema deo,Anand,1971


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [12]:
%%time
# Write your sql query below

query = """
        SELECT name FROM person 
        WHERE pid in
        (
        SELECT mc.pid FROM m_cast mc JOIN movie m ON mc.mid=m.mid WHERE m.year<1970
        INTERSECT
        SELECT mc.pid FROM m_cast mc JOIN movie m ON mc.mid=m.mid WHERE m.year>1990
        )
        
        """

q3 = pd.read_sql_query(query, conn)    
print(q3.shape)

(300, 1)
Wall time: 389 ms


In [13]:
q3.head()

,Name
0,rishi kapoor
1,amitabh bachchan
2,asrani
3,zohra sehgal
4,parikshat sahni


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [14]:
%%time
# Write your sql query below

query = """
        SELECT DISTINCT(p.name) ,count(m.mid) AS number_of_movies_directed FROM person p
        LEFT JOIN m_director md ON p.pid=md.pid
        LEFT JOIN movie m ON m.mid=md.mid
        GROUP BY p.name
        HAVING COUNT(m.mid)>=10
        ORDER BY number_of_movies_directed DESC
        """

q4 = pd.read_sql_query(query, conn)    
print(q4.shape)

(58, 2)
Wall time: 102 ms


In [15]:
q4.head()

,Name,number_of_movies_directed
0,david dhawan,39
1,mahesh bhatt,36
2,priyadarshan,30
3,ram gopal varma,30
4,vikram bhatt,29


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [16]:
%%time
# Write your sql query below

query = """
        SELECT m.year,m.title,COUNT(*) AS onle_femal_movies FROM movie m 
        JOIN(
        SELECT DISTINCT mid FROM m_cast 
        WHERE mid NOT IN
        (SELECT mc.mid FROM m_cast mc JOIN person p ON p.pid=mc.pid WHERE p.gender ='Male'
        UNION
        SELECT mc.mid FROM m_cast mc WHERE mc.pid IS NULL)) fm 
        ON fm.mid = m.mid
        GROUP BY m.year
        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)

(4, 3)
Wall time: 358 ms


In [17]:
q5a.head(7)

,year,title,onle_femal_movies
0,1939,Kala Jigar,1
1,1999,Bindhaast,1
2,2000,Snegithiye,1
3,2018,Vaibhav Sethia: Don't,2


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [18]:
%%time
# Write your sql query below

query = """
        SELECT m.year,COUNT(m.mid) AS total_movies_in_year,f.only_female_movies,(f.only_female_movies*100/(COUNT(m.mid)*1.0))
        FROM movie m
        LEFT JOIN (SELECT m.year,COUNT(*) AS only_female_movies FROM movie m 
        JOIN(
        SELECT DISTINCT mid FROM m_cast 
        WHERE mid NOT IN
        (SELECT mc.mid FROM m_cast mc JOIN person p ON p.pid=mc.pid WHERE p.gender ='Male'
        UNION
        SELECT mc.mid FROM m_cast mc WHERE mc.pid IS NULL))
        fm ON fm.mid = m.mid
        GROUP BY m.year) f ON f.year = m.year 
        GROUP BY m.year
        
        """

q5b = pd.read_sql_query(query, conn)    
print(q5b.shape)

(78, 4)
Wall time: 361 ms


In [19]:
q5b

,year,total_movies_in_year,only_female_movies,(f.only_female_movies*100/(COUNT(m.mid)*1.0))
0,1931,1,NaN,NaN
1,1936,3,NaN,NaN
2,1939,2,1.0,50.000000
3,1941,1,NaN,NaN
4,1943,1,NaN,NaN
5,1946,2,NaN,NaN
6,1947,2,NaN,NaN
7,1948,3,NaN,NaN
8,1949,3,NaN,NaN
9,1950,2,NaN,NaN


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [20]:
%%time
# Write your sql query below

query = """
        SELECT name,MAX(title) AS largest_cast_size FROM 
        (SELECT m.title AS name, COUNT(DISTINCT(mc.pid)) AS title FROM m_cast AS mc 
        LEFT JOIN movie m ON mc.mid=m.mid GROUP BY m.mid)
        
        """

q6 = pd.read_sql_query(query, conn)    
print(q6.shape)

(1, 2)
Wall time: 174 ms


In [21]:
q6.head()

,name,largest_cast_size
0,Ocean's Eight,238


## Q7 --- A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [22]:
%%time
# Write your sql query below

query = """
        SELECT DISTINCT m.year AS decade_start,m.year+9 AS decade_end,COUNT(DISTINCT(y.mid)) AS no_of_movies
        FROM movie m,movie y 
        WHERE y.year >= decade_Start and y.year <=decade_end
        GROUP BY decade_start
        ORDER BY no_of_movies DESC
        LIMIT 1
        """

q7 = pd.read_sql_query(query, conn)    
print(q7.shape)

(1, 3)
Wall time: 5.81 s


In [23]:
q7

,decade_start,decade_end,no_of_movies
0,2008,2017,1205


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [24]:
%%time
# Write your sql query below

query = """
       SELECT y.Actor,y.yash_chopra_movies,ny.NON_yash_chopra_movies FROM
       (SELECT DISTINCT(p.name) AS Actor,p1.name AS Director,COUNT(m.title) AS yash_chopra_movies FROM person p
       join m_cast mc ON p.pid=mc.pid
       join movie m ON m.mid=mc.mid
       JOIN m_director md ON md.mid=m.mid
       JOIN person p1 ON p1.pid=md.pid
       WHERE p1.name ='yash chopra'
       GROUP BY p.name)y
       LEFT join
       (SELECT DISTINCT(p.name) AS Actor,p1.name AS Director,COUNT(m.title) AS NON_yash_chopra_movies FROM person p
       join m_cast mc ON p.pid=mc.pid
       join movie m ON m.mid=mc.mid
       JOIN m_director md ON md.mid=m.mid
       JOIN person p1 ON p1.pid=md.pid
       WHERE p1.name !='yash chopra'
       GROUP BY p.name)ny ON y.Actor = ny.Actor
       WHERE y.yash_chopra_movies>ny.NON_yash_chopra_movies OR ny.NON_yash_chopra_movies is null
        """

q8 = pd.read_sql_query(query, conn)    
print(q8.shape)

(85, 3)
Wall time: 866 ms


In [25]:
q8.head(100)

,Actor,yash_chopra_movies,NON_yash_chopra_movies
0,abbie murphy,1,NaN
1,akhtar-ul-iman,1,NaN
2,amarinder sodhi,2,NaN
3,anick wiget,1,NaN
4,ashok chadda,1,NaN
5,ashok verma,2,1.0
6,benjayx murphy,1,NaN
7,camilla talarowska,1,NaN
8,celine jedidi,1,NaN
9,chandni jas keerat,1,NaN


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [26]:
%%time
# Write your sql query below

query = """
        SELECT DISTINCT p.pid,p.name FROM person p JOIN m_cast mc ON mc.pid=p.pid WHERE p.name !='shah rukh khan'
        AND mid IN
        (SELECT mc.mid FROM m_cast mc WHERE pid in
        (SELECT p.pid FROM person p JOIN m_cast mc ON mc.pid=p.pid WHERE p.name !='shah rukh khan'
        AND mid IN
        (SELECT mc.mid FROM m_cast mc JOIN person p ON p.pid=mc.pid WHERE p.name='shah rukh khan')))
        AND p.pid NOT IN
        (SELECT p.pid FROM person p JOIN m_cast mc ON mc.pid=p.pid WHERE p.name !='shah rukh khan'
        AND mid IN
        (SELECT mc.mid FROM m_cast mc JOIN person p ON p.pid=mc.pid WHERE p.name='shah rukh khan'))
        """

q9 = pd.read_sql_query(query, conn)    
print(q9.shape)

(25698, 2)
Wall time: 1.84 s


In [27]:
q9.head()

,PID,Name
0,nm2951768,freida pinto
1,nm4575116,rohan chand
2,nm0949433,damian young
3,nm1753302,waris ahluwalia
4,nm6467532,caroline christl long
